# Building an ETL Pipeline

As the second part of the predict for Gather, you will need to build a pipeline of functions in python which does the following:

1. Function to connect to twitter and scrapes "Eskom_SA" tweets.
<br>
<br>
2. Cleans/Processes the tweets from the scraped tweets which will create a dataframe with two new columns using the following functions: <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; a) Hashtag Remover from Analyse Functions
<br>
<br>
3. Functions which connects to your SQL database and uploads the tweets into the table you store the tweets in the database.

In [1]:
# General:
import tweepy           # To consume Twitter's API
import pandas as pd     # To handle data
import numpy as np      # For numerical computation
import json
# For plotting and visualization:
from IPython.display import display
import pyodbc


# Consumer and Access details

Fill in your Consumer and Access details you should have recieved when applying for a Twitter API. 

In [2]:
# Consumer:
#CONSUMER_KEY    =  # Consumer Key
#CONSUMER_SECRET =  # Consumer Secret

# Access:
#ACCESS_TOKEN  =    # Access Token
#ACCESS_SECRET =    # Access Secret

In [3]:
# API's setup:
def twitter_setup():
    """
    Utility function to setup the Twitter's API
    with access and consumer keys from Twitter.
    """

    # Authentication and access using keys:
    auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
    auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)

    # Return API with authentication:
    api = tweepy.API(auth, timeout=1000)
    return api

# Function 1:

Write a function which:
- Scrapes _"Eskom_SA"_ tweets from Twitter. 

Function Specifications:
- The function should return a dataframe with the scraped tweets with just the "_Tweets_" and "_Date_". 
- Will take in the ```consumer key,  consumer secret code, access token``` and ```access secret code```.

NOTE:
The dataframe should have the same column names as those in your SQL Database table where you store the tweets.

In [4]:
def twitter_df(CONSUMER_KEY, CONSUMER_SECRET, ACCESS_TOKEN, ACCESS_SECRET ):

    api = twitter_setup()
    all_tweets = []
    tweets = api.user_timeline(screen_name='ESKOM_SA', 
                           # 200 is the maximum allowed count
                           count=200,
                           include_rts = False,
                           # Necessary to keep full_text 
                           # otherwise only the first 140 words are extracted
                           tweet_mode = 'extended'
                           )
    all_tweets.extend(tweets)
  
    outtweets = [[tweet.full_text.encode("utf-8").decode("utf-8"),tweet.created_at] 
              for idx,tweet in enumerate(all_tweets)]
    df = pd.DataFrame(outtweets,columns=['Tweets','Date'])
    df.set_index('Tweets')
    return df

# Function 2: Removing hashtags and the municipalities

Write a function which:
- Uses the function you wrote in the Analyse section to extract the hashtags and municipalities into it's own column in a new data frame. 

Function Specifications:
- The function should take in the pandas dataframe you created in Function 1 and return a new pandas dataframe. 

In [6]:
twitter_df(CONSUMER_KEY, CONSUMER_SECRET, ACCESS_TOKEN, ACCESS_SECRET)

In [ ]:
def extract_municipality_hashtags(df):
    
     mun_dict = {'@CityofCTAlerts' : 'Cape Town',
                '@CityPowerJhb' : 'Johannesburg',
                '@eThekwiniM' : 'eThekwini' ,
                '@EMMInfo' : 'Ekurhuleni',
                '@centlecutility' : 'Mangaung',
                '@NMBmunicipality' : 'Nelson Mandela Bay',
                '@CityTshwane' : 'Tshwane'}
    
    handles = list(mun_dict.keys())
    
    df['municipality'] = df['Tweets'].str.extract('({})'.format('|'.join(handles)), expand=False).fillna(np.nan)
    df['municipality'] = df['municipality'].map(mun_dict)
    
    df['hashtags'] = df['Tweets'].str.findall(r'#.*?(?=\s|$)')
    df['hashtags'] = df['hashtags'].apply(lambda x: np.nan if len(x)==0 else [x.lower() for x in x])
    
    return df

# Function 3: Updating SQL Database with pyODBC

Write a function which:
- Connects and updates your SQL database. 

Function Specifications:
- The function should take in a pandas dataframe created in Function 2. 
- Connect to your SQL database.
- Update the table you store your tweets in.
- Not return any output.

In [ ]:
def pyodbc_twitter(connection, df, twitter_table):
    """ Connects to a SQL database and updates the twitter table on the database with tweets from a give dataframe

    Parameters:
      connection(object): The object that contains authentication and connection details to the SQL Database
      df(dataframe): Pandas dataframe that contains tweets to be appended to the SQL database
      twitter_table(object): Table object that will be updated with tweets from the dataframe

    Returns:
      None: Does not return any output
    """
    conn = connection
    c = conn.cursor()

    c.execute(twitter_table)
    conn.commit()

    df.to_sql('twitter_table', conn, if_exists='replace', index = False)
    print('Database Updated Successfully')
    return None

In [ ]:
pyodbc_twitter(connection = pyodbc.connect(driver='{SQL Server}',
                      host='EDSA-HC9M9S2\SQLEXPRESS',
                      database='gather_eskom',
                      trusted_connection='true',
                      user='sa'),
    df = extract_municipality_hashtags(twitter_df(CONSUMER_KEY, CONSUMER_SECRET, ACCESS_TOKEN, ACCESS_SECRET)),
    twitter_table = 'CREATE TABLE twitter_table (Tweets text, Date text, municipality text, hashtags text)')